In [1]:
from simulator.sim3 import *
from exploration.env.func import Experiment, Env
from exploration.random.func import RANDOM
import numpy as np
from codegeneration import generate_instruction_sequence
from simulator.sim3 import print_contention_analysis
import pandas as pd

# Perform tests for the simulator

In [30]:
print("===================================================================")
print("1 core, 2 read cycles, same cache line, no dependency")
print("===================================================================")
# 1st RD => cache miss => DDR reads transaction 
# 2nd RD => cache hit => no DDR transaction
# Create instruction sequences
GlobalVar.clear_history()
inst0 = { 0: ('read', 0), 60: ('read', 17) }
inst1 = {}
inst1 = { 0:('read', 4), 1:('read',0)}

exp=Experiment()
exp.load_instr(inst0, inst1)

results = exp.simulate(200, display_stats=True)


1 core, 2 read cycles, same cache line, no dependency

--- Simulation Stats ---
{'core': 0, 'L1': {'level': 'L1', 'hits': 0, 'misses': 2, 'miss_rate': 1.0}, 'L2': {'level': 'L2', 'hits': 0, 'misses': 4, 'miss_rate': 1.0}}
{'core': 1, 'L1': {'level': 'L1', 'hits': 0, 'misses': 2, 'miss_rate': 1.0}, 'L2': {'level': 'L2', 'hits': 0, 'misses': 4, 'miss_rate': 1.0}}


In [3]:
results['shared_resource_events']

[{'cycle': 0,
  'type': 'L2_CACHE_CONTENTION',
  'resource': 'L2_CACHE',
  'initiators': [0, 1],
  'details': {'set_indices': [0, 1],
   'operations': ['read', 'read'],
   'addresses': [0, 4]}},
 {'cycle': 6,
  'type': 'DDR_MEMORY_CONTENTION',
  'resource': 'DDR_MEMORY',
  'initiators': [0, 1, 1],
  'details': {'banks': [0, 0, 0],
   'rows': [0, 0, 0],
   'operations': ['read', 'read', 'read'],
   'statuses': ['ROW MISS', 'waiting', 'waiting'],
   'bank_conflicts': True,
   'row_conflicts': False}}]

In [4]:
results['contention_events']

{'l2_contention_cycles': [0],
 'ddr_contention_cycles': [6],
 'total_contention_events': 2}

In [5]:
print("===================================================================")
print("1 core, 2 read cycles, same cache line, no dependency")
print("===================================================================")
# 1st RD => cache miss => DDR reads transaction 
# 2nd RD => cache hit => no DDR transaction
# Create instruction sequences
GlobalVar.clear_history()
inst0 = { 0: ('read', 0), 60: ('read', 17) }
inst1 = {}
#inst1 = { 0:('read', 4)}

exp=Experiment()
exp.load_instr(inst0, inst1)

results = exp.simulate(200, display_stats=True)


1 core, 2 read cycles, same cache line, no dependency

--- Simulation Stats ---
{'core': 0, 'L1': {'level': 'L1', 'hits': 0, 'misses': 2, 'miss_rate': 1.0}, 'L2': {'level': 'L2', 'hits': 0, 'misses': 2, 'miss_rate': 1.0}}
{'core': 1, 'L1': {'level': 'L1', 'hits': 0, 'misses': 0, 'miss_rate': 0}, 'L2': {'level': 'L2', 'hits': 0, 'misses': 2, 'miss_rate': 1.0}}


In [6]:
print("===================================================================")
print("1 core, 2 read cycles, same cache line, no dependency")
print("===================================================================")
# 1st RD => cache miss => DDR reads transaction 
# 2nd RD => cache hit => no DDR transaction
# Create instruction sequences
inst0 = { 0: ('read', 0), 60: ('read', 2) }
inst1 = {  }

exp=Experiment()
exp.load_instr(inst0, inst1)

results = exp.simulate(100, display_stats=True)

1 core, 2 read cycles, same cache line, no dependency

--- Simulation Stats ---
{'core': 0, 'L1': {'level': 'L1', 'hits': 0, 'misses': 2, 'miss_rate': 1.0}, 'L2': {'level': 'L2', 'hits': 0, 'misses': 2, 'miss_rate': 1.0}}
{'core': 1, 'L1': {'level': 'L1', 'hits': 0, 'misses': 0, 'miss_rate': 0}, 'L2': {'level': 'L2', 'hits': 0, 'misses': 2, 'miss_rate': 1.0}}


In [7]:
print(results['contention_events'])

{'l2_contention_cycles': [], 'ddr_contention_cycles': [], 'total_contention_events': 0}


In [68]:
print(results['shared_resource_events'])

[{'cycle': 0, 'type': 'L2_CACHE_CONTENTION', 'resource': 'L2_CACHE', 'initiators': [0, 1], 'details': {'set_indices': [0, 1], 'operations': ['read', 'read'], 'addresses': [0, 4]}}]


In [31]:
GlobalVar.clear_history()
print("===================================================================")
print("1 core, 2 read cycles, different bank, no dependency")
print("===================================================================")
# bank nb = addr % num_banks, with num_banks=4
# row nb = addr // 16
# 1st RD in bank 0, row 0 
# 1st RD in bank 1, row 1
# 2nd RD => cache hit => no DDR transaction
# Create instruction sequences

# 1st RD => cache miss => DDR reads transaction 
# 2nd RD => cache miss => DDR reads transaction 
inst0 = { 0: ('read', 0)}
inst1 = {  }

exp=Experiment()
exp.load_instr(inst0, inst1)

exp.simulate(100)

1 core, 2 read cycles, different bank, no dependency


{'time_core0': 51,
 'time_core1': 0,
 'miss_ratios_detailled': array([[ 1., -0., -0., -0.],
        [-0., -0., -0., -0.]]),
 'miss_ratios_global': np.float64(1.0),
 'L1_miss_ratio_core0': 1.0,
 'L1_miss_ratio_core1': 0,
 'L2_miss_ratio': 1.0,
 'contention_events': {'l2_contention_cycles': [],
  'ddr_contention_cycles': [],
  'total_contention_events': 0},
 'shared_resource_events': []}

In [32]:
GlobalVar.clear_history()
print("===================================================================")
print("1 core, 2 read cycles, different bank, no dependency")
print("===================================================================")
# bank nb = addr % num_banks, with num_banks=4
# row nb = addr // 16
# 1st RD in bank 0, row 0 
# 1st RD in bank 1, row 1
# 2nd RD => cache hit => no DDR transaction
# Create instruction sequences

# 1st RD => cache miss => DDR reads transaction 
# 2nd RD => cache miss => DDR reads transaction 
inst0 = { 0: ('read', 0), 60: ('read', 17)}
inst1 = {  }

exp=Experiment()
exp.load_instr(inst0, inst1)

exp.simulate(100)

1 core, 2 read cycles, different bank, no dependency


{'time_core0': 112,
 'time_core1': 0,
 'miss_ratios_detailled': array([[ 1., -0., -0., -0.],
        [-0.,  1., -0., -0.]]),
 'miss_ratios_global': np.float64(1.0),
 'L1_miss_ratio_core0': 1.0,
 'L1_miss_ratio_core1': 0,
 'L2_miss_ratio': 1.0,
 'contention_events': {'l2_contention_cycles': [],
  'ddr_contention_cycles': [],
  'total_contention_events': 0},
 'shared_resource_events': []}

We look at the miss ratios in cache and memory:

In [2]:
print('One miss in the ddr memory, row 0, bank 0')
def visu_ddr(tab):
    df = pd.DataFrame(tab,columns = [f'bank {j}' for j in range(4)], index = [f'row {j}' for j in range(2)])
    return df
visu_ddr(exp.miss_tab)

One miss in the ddr memory, row 0, bank 0


NameError: name 'exp' is not defined

In [15]:
print('No hit occurs')
visu_ddr(exp.hits_tab)

No hit occurs


,bank 0,bank 1,bank 2,bank 3
row 0,0.0,0.0,0.0,0.0
row 1,0.0,0.0,0.0,0.0


# What data do we track ? 

In [66]:
p0 = generate_instruction_sequence(50)
p1 = generate_instruction_sequence(50)

In [67]:
GlobalVar.clear_history()
E =Env(300)
out = E({'core0':p0, 'core1':p1})

In [75]:
def subsequence(cycle,parameter):
    '''
    returns the sequences of instructions up to the given cycle
    '''
    return {'core0':{k:parameter['core0'][k] for k in parameter['core0'] if k<=cycle},
     'core1':{k:parameter['core1'][k] for k in parameter['core1'] if k<cycle}}

In [68]:
out['mutual']['shared_resource_events']

[{'cycle': 0,
  'type': 'L2_CACHE_CONTENTION',
  'resource': 'L2_CACHE',
  'initiators': [0, 1],
  'details': {'set_indices': [1, 3],
   'operations': ['read', 'read'],
   'addresses': [5, 12]}},
 {'cycle': 1,
  'type': 'L2_CACHE_CONTENTION',
  'resource': 'L2_CACHE',
  'initiators': [0, 1],
  'details': {'set_indices': [4, 4],
   'operations': ['read', 'read'],
   'addresses': [17, 17]}},
 {'cycle': 6,
  'type': 'L2_CACHE_CONTENTION',
  'resource': 'L2_CACHE',
  'initiators': [0, 1],
  'details': {'set_indices': [0, 4],
   'operations': ['read', 'read'],
   'addresses': [3, 18]}},
 {'cycle': 27,
  'type': 'DDR_MEMORY_CONTENTION',
  'resource': 'DDR_MEMORY',
  'initiators': [1, 1, 0, 1],
  'details': {'banks': [2, 0, 2, 2],
   'rows': [1, 0, 0, 0],
   'operations': ['read', 'read', 'read', 'read'],
   'statuses': ['ROW HIT', 'waiting', 'waiting', 'waiting'],
   'bank_conflicts': True,
   'row_conflicts': True}},
 {'cycle': 31,
  'type': 'DDR_MEMORY_CONTENTION',
  'resource': 'DDR_MEMOR

In [72]:
def shared_ressource2vec(in_,E):
    count_banks = np.histogram(in_['details']['banks'],bins = range(E.num_banks+1))[0]/len(in_['details']['banks'])
    count_rows = np.histogram(in_['details']['rows'],bins = range(E.num_rows+1))[0]/len(in_['details']['banks'])
    ratios_core = np.array([sum(np.array(in_['initiators'])==1)/len(in_['initiators'])])
    conflicts = np.array([1*in_['details']['bank_conflicts'],1*in_['details']['row_conflicts']])
    out = np.concatenate((ratios_core,count_banks,count_rows,conflicts),axis=0)
    return out

In [64]:
out_shared = shared_ressource2vec(out['mutual']['shared_resource_events'][1],E) 

In [65]:
out_shared

array([0.5, 1. , 0. , 0. , 0. , 1. , 0. , 1. , 0. ])

In [58]:
GlobalVar.clear_history()
exp=Experiment()
exp.load_instr(p0, p1)
results = exp.simulate(100, display_stats=False)
GlobalVar.clear_history()
print(results)

add candidates info for ddr
add candidates info for ddr
add candidates info for ddr
{'time_core0': 64, 'time_core1': 57, 'miss_ratios_detailled': array([[ 1. ,  0.2,  1. ,  0.5],
       [-0. , -0. , -0. , -0. ]]), 'miss_ratios_global': np.float64(0.16666666666666666), 'L1_miss_ratio_core0': 0.3333333333333333, 'L1_miss_ratio_core1': 1.0, 'L2_miss_ratio': 1.0, 'contention_events': {'l2_contention_cycles': [5], 'ddr_contention_cycles': [], 'total_contention_events': 1}, 'shared_resource_events': [{'cycle': 5, 'type': 'L2_CACHE_CONTENTION', 'resource': 'L2_CACHE', 'initiators': [0, 1], 'details': {'set_indices': [1, 1], 'operations': ['read', 'read'], 'addresses': [5, 7]}}]}


In [53]:
results

{'time_core0': 62,
 'time_core1': 56,
 'miss_ratios_detailled': array([[ 1. ,  0.2,  1. ,  0.5],
        [-0. , -0. , -0. , -0. ]]),
 'miss_ratios_global': np.float64(0.16666666666666666),
 'L1_miss_ratio_core0': 0.3333333333333333,
 'L1_miss_ratio_core1': 1.0,
 'L2_miss_ratio': 1.0,
 'contention_events': {'l2_contention_cycles': [5],
  'ddr_contention_cycles': [],
  'total_contention_events': 1},
 'shared_resource_events': [{'cycle': 5,
   'type': 'L2_CACHE_CONTENTION',
   'resource': 'L2_CACHE',
   'initiators': [0, 1],
   'details': {'set_indices': [1, 1],
    'operations': ['read', 'read'],
    'addresses': [5, 7]}}]}

In [44]:
out['mutual']

{'time_core0': 0,
 'time_core1': 0,
 'miss_ratios_detailled': array([[-0., -0., -0., -0.],
        [-0., -0., -0., -0.]]),
 'miss_ratios_global': 0,
 'L1_miss_ratio_core0': 0,
 'L1_miss_ratio_core1': 0,
 'L2_miss_ratio': 0,
 'contention_events': {'l2_contention_cycles': [],
  'ddr_contention_cycles': [],
  'total_contention_events': 0},
 'shared_resource_events': []}

In [8]:
GlobalVar.l2_access_log

[]

# We conceive an History to enrich during exploration

In [8]:
from exploration.history import History

In [9]:
H = History()

In [10]:
entry = {'program':{'core0':[p0],'core1':[p1]}} | out

In [11]:
entry['mutual']['time_core0']

0

In [12]:
entry['mutual']['shared_resource_events']

[]

In [13]:
H.store(entry)

In [14]:
H.memory_perf['mutual'].keys()

dict_keys(['time_core0', 'time_core1', 'miss_ratios_detailled', 'miss_ratios_global', 'L1_miss_ratio_core0', 'L1_miss_ratio_core1', 'L2_miss_ratio', 'contention_events', 'shared_resource_events'])

In [15]:
H.memory_perf['mutual']['shared_resource_events']

[[]]

# Perform exploration with random generated programs

In [2]:
from exploration.history import History

In [3]:
N = 201

E =Env(300)
H = History(env=E)
random_explor = RANDOM(N,E,H)

In [4]:
random_explor()

step 0/200
step 100/200
step 200/200


In [5]:
from exploration.imgep.OptimizationPolicy import OptimizationPolicykNN as OP

In [16]:
Pi = OP(k=2)

In [17]:
help(Pi)

Help on OptimizationPolicykNN in module exploration.imgep.OptimizationPolicy object:

class OptimizationPolicykNN(exploration.imgep.features.Features)
 |  OptimizationPolicykNN(k=1, num_mutations=1, max_len=50, num_addr=20, num_bank=4, min_instr=5, max_instr=50, max_cycle=60, min_address_core0=0, max_address_core0=10, min_address_core1=11, max_address_core1=21)
 |  
 |  Method resolution order:
 |      OptimizationPolicykNN
 |      exploration.imgep.features.Features
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(self, goal: numpy.ndarray, H: exploration.history.History, module: int) -> dict
 |      Call self as a function.
 |  
 |  __init__(self, k=1, num_mutations=1, max_len=50, num_addr=20, num_bank=4, min_instr=5, max_instr=50, max_cycle=60, min_address_core0=0, max_address_core0=10, min_address_core1=11, max_address_core1=21)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  feature2closest_code(self, features, signature: num

In [18]:
p0 = generate_instruction_sequence(50)
p1 = generate_instruction_sequence(50)
programs = {'core0':p0,'core1':p1}

In [19]:
t = GlobalVar.global_cycle
t=2

In [20]:
print(H.memory_perf.keys())

dict_keys(['mutual', 'core0', 'core1'])


In [21]:
H.memory_perf['mutual'].keys()

dict_keys(['time_core0', 'time_core1', 'miss_ratios_detailled', 'miss_ratios_global', 'L1_miss_ratio_core0', 'L1_miss_ratio_core1', 'L2_miss_ratio', 'shared_resource_events'])

In [22]:
feature = H.as_array()[0]

In [12]:
print(GlobalVar.global_cycle)

0


In [13]:
Pi.feature2closest_code(feature,np.array([62]))

array([2])

In [27]:
Pi(np.array([62]),H,0)

{'core0': {0: ('write', 0),
  6: ('read', 8),
  9: ('read', 8),
  11: ('write', 2),
  12: ('read', 1),
  14: ('write', 1)},
 'core1': {0: ('write', 17),
  7: ('read', 11),
  12: ('write', 19),
  18: ('read', 20),
  20: ('read', 13),
  22: ('read', 15),
  26: ('write', 20),
  31: ('read', 17),
  34: ('read', 18),
  36: ('write', 18),
  41: ('read', 15),
  43: ('read', 17),
  46: ('read', 12),
  54: ('write', 19)}}

In [14]:
help(Pi.__call__)

Help on method __call__ in module exploration.imgep.OptimizationPolicy:

__call__(goal: numpy.ndarray, H: exploration.history.History, module: int) -> dict method of exploration.imgep.OptimizationPolicy.OptimizationPolicykNN instance
    Call self as a function.



In [34]:
features[88]

np.float64(63.0)

In [28]:
print([len(t) for t in tab])

[201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201]


In [18]:
module = {'type':'vec_miss_ratios_detailled','core':'mutual','bank':0,'row':0}

In [19]:
F.data2feature(H.memory_perf, module).shape

(201, 2, 4)

In [34]:
from simulator.sim3 import GlobalVar

In [12]:
GlobalVar.global_cycle

0

In [26]:
GlobalVar.clear_history()
print("=============================================================================================================")
print("1 core, 2 read cycles,same index, same tag, same bank, different rows, no dependency")
print("=============================================================================================================")
# 1st RD => cache miss => DDR reads transaction 
# 2nd RD => cache miss => DDR reads transaction 
# Create instruction sequences
inst0 = { 0: ('read', 0), 60: ('read', 20) }
inst0 = {60: ('read', 20),  0: ('read', 0)}
inst1 = {}
exp=Experiment()
exp.load_instr(inst0, inst0)
exp.simulate(200)

1 core, 2 read cycles,same index, same tag, same bank, different rows, no dependency

--- Simulation Stats ---
{'core': 0, 'L1': {'level': 'L1', 'hits': 0, 'misses': 2, 'miss_rate': 1.0}, 'L2': {'level': 'L2', 'hits': 0, 'misses': 4, 'miss_rate': 1.0}}
{'core': 1, 'L1': {'level': 'L1', 'hits': 0, 'misses': 2, 'miss_rate': 1.0}, 'L2': {'level': 'L2', 'hits': 0, 'misses': 4, 'miss_rate': 1.0}}


{'time_core0': 111,
 'time_core1': 81,
 'miss_ratios_detailled': array([[ 0.5, -0. , -0. , -0. ],
        [ 0.5, -0. , -0. , -0. ]]),
 'miss_ratios_global': np.float64(0.5),
 'L1_miss_ratio_core0': 1.0,
 'L1_miss_ratio_core1': 1.0,
 'L2_miss_ratio': 1.0}

In [35]:
print_contention_analysis()


=== SHARED RESOURCE CONTENTION ANALYSIS ===
Total contention events: 2
L2 cache contention cycles: 2
DDR memory contention cycles: 0

=== DETAILED CONTENTION EVENTS ===
Cycle 200: L2_CACHE_CONTENTION
  Cores involved: [0, 1]
  Details: {'set_indices': [0, 0, 0, 0], 'operations': ['read', 'read', 'read', 'read'], 'addresses': [0, 0, 0, 0]}

Cycle 200: L2_CACHE_CONTENTION
  Cores involved: [0, 1]
  Details: {'set_indices': [5, 5, 5, 5], 'operations': ['read', 'read', 'read', 'read'], 'addresses': [20, 20, 20, 20]}



In [27]:
GlobalVar.l2_access_log

[{'cycle': 2,
  'core_id': 1,
  'addr': 14,
  'operation': 'read',
  'set_index': 3,
  'way': -1,
  'hit': False},
 {'cycle': 2,
  'core_id': 1,
  'addr': 14,
  'operation': 'read',
  'set_index': 3,
  'way': -1,
  'hit': False},
 {'cycle': 52,
  'core_id': 1,
  'addr': 12,
  'operation': 'write',
  'set_index': 3,
  'way': -1,
  'hit': False},
 {'cycle': 52,
  'core_id': 1,
  'addr': 12,
  'operation': 'write',
  'set_index': 3,
  'way': 0,
  'hit': True}]

In [23]:
def print_cache_states_history():
    """Print the cache states history in a readable format"""
    print("\n=== CACHE STATES HISTORY ===")
    for state in GlobalVar.cache_states_history:
        print(f"\nCycle {state['cycle']}: Core {state['core_id']} {state['level']} {state['operation'].upper()} @ {state['address']}")
        
        for set_state in state['sets']:
            set_index = set_state['set_index']
            #print(f"  Set {set_index}:")
            
            for line_state in set_state['lines']:
                status = "INVALID"
                if line_state['valid']:
                    status = "CLEAN" if not line_state['dirty'] else "DIRTY"
                
                tag_info = f"Tag: {line_state['tag']}" if line_state['tag'] is not None else "Tag: None"
                #print(f"    Line {line_state['line_index']}: {status} | {tag_info}")

def get_cache_states_summary():
    """Get a summary of cache states history"""
    summary = []
    for state in GlobalVar.cache_states_history:
        summary.append({
            'cycle': state['cycle'],
            'core_id': state['core_id'], 
            'level': state['level'],
            'operation': state['operation'],
            'address': state['address'],
            'total_sets': len(state['sets']),
            'total_lines': sum(len(set_state['lines']) for set_state in state['sets'])
        })
    return summary

In [13]:
# Print full history
print_cache_states_history()

## Get summary
#summary = get_cache_states_summary()
#for entry in summary:
#    print(f"Cycle {entry['cycle']}: Core {entry['core_id']} {entry['level']} {entry['operation']} @ {entry['address']}")
#
## Access raw data
#for state in GlobalVar.cache_states_history:
#    print(f"Cycle {state['cycle']} - Operation: {state['operation']}")

NameError: name 'print_cache_states_history' is not defined

In [31]:
GlobalVar.clear_history()

In [25]:
# Helper functions to analyze state changes
def print_state_changes():
    """Print all state changes in a readable format"""
    print("\n=== CACHE STATE CHANGES ===")
    
    if not GlobalVar.cache_state_changes:
        print("No state changes detected.")
        return
    
    for change in GlobalVar.cache_state_changes:
        print(f"\nCycle {change['cycle']}: Core {change['core_id']} {change['level']} {change['operation'].upper()} @ {change['address']}")
        print(f"  Set {change['set_index']}, Line {change['line_index']}:")
        print(f"    Valid: {change['previous_valid']} → {change['current_valid']}")
        print(f"    Dirty: {change['previous_dirty']} → {change['current_dirty']}")
        print(f"    Tag: {change['previous_tag']} → {change['current_tag']}")
        print(f"    Changes: {', '.join([f'{c[0]}: {c[1]}' for c in change['changes']])}")

def get_valid_transitions():
    """Get all valid bit transitions"""
    valid_changes = [c for c in GlobalVar.cache_state_changes 
                    if any(change[0] == 'valid' for change in c['changes'])]
    
    transitions = {}
    for change in valid_changes:
        for ch_type, ch_desc in change['changes']:
            if ch_type == 'valid':
                if ch_desc not in transitions:
                    transitions[ch_desc] = 0
                transitions[ch_desc] += 1
    
    return transitions

def get_dirty_transitions():
    """Get all dirty bit transitions"""
    dirty_changes = [c for c in GlobalVar.cache_state_changes 
                    if any(change[0] == 'dirty' for change in c['changes'])]
    
    transitions = {}
    for change in dirty_changes:
        for ch_type, ch_desc in change['changes']:
            if ch_type == 'dirty':
                if ch_desc not in transitions:
                    transitions[ch_desc] = 0
                transitions[ch_desc] += 1
    
    return transitions

def get_state_change_summary():
    """Get a summary of all state changes"""
    summary = {
        'total_changes': len(GlobalVar.cache_state_changes),
        'valid_transitions': get_valid_transitions(),
        'dirty_transitions': get_dirty_transitions(),
        'changes_by_level': {},
        'changes_by_operation': {},
        'changes_by_core': {}
    }
    
    # Group by level
    for change in GlobalVar.cache_state_changes:
        level = change['level']
        if level not in summary['changes_by_level']:
            summary['changes_by_level'][level] = 0
        summary['changes_by_level'][level] += 1
    
    # Group by operation
    for change in GlobalVar.cache_state_changes:
        op = change['operation']
        if op not in summary['changes_by_operation']:
            summary['changes_by_operation'][op] = 0
        summary['changes_by_operation'][op] += 1
    
    # Group by core
    for change in GlobalVar.cache_state_changes:
        core = change['core_id']
        if core not in summary['changes_by_core']:
            summary['changes_by_core'][core] = 0
        summary['changes_by_core'][core] += 1
    
    return summary

def print_state_change_summary():
    """Print a summary of state changes"""
    summary = get_state_change_summary()
    
    print("\n=== STATE CHANGE SUMMARY ===")
    print(f"Total state changes: {summary['total_changes']}")
    
    print("\nBy Cache Level:")
    for level, count in summary['changes_by_level'].items():
        print(f"  {level}: {count} changes")
    
    print("\nBy Operation:")
    for op, count in summary['changes_by_operation'].items():
        print(f"  {op}: {count} changes")
    
    print("\nBy Core:")
    for core, count in summary['changes_by_core'].items():
        print(f"  Core {core}: {count} changes")
    
    print("\nValid Bit Transitions:")
    for transition, count in summary['valid_transitions'].items():
        print(f"  {transition}: {count}")
    
    print("\nDirty Bit Transitions:")
    for transition, count in summary['dirty_transitions'].items():
        print(f"  {transition}: {count}")


In [29]:
def example_usage():
    """Example of how to use the state change tracking"""
    # Run your simulation...
    
    # After simulation, analyze changes:
    print_state_changes()  # Print all individual changes
    print_state_change_summary()  # Print summary
    
    # Get specific transitions:
    valid_transitions = get_valid_transitions()
    dirty_transitions = get_dirty_transitions()
    
    print(f"\nValid→Invalid transitions: {valid_transitions.get('VALID→INVALID', 0)}")
    print(f"Invalid→Valid transitions: {valid_transitions.get('INVALID→VALID', 0)}")
    print(f"Clean→Dirty transitions: {dirty_transitions.get('CLEAN→DIRTY', 0)}")
    print(f"Dirty→Clean transitions: {dirty_transitions.get('DIRTY→CLEAN', 0)}")
    
    # Access raw change data:
    for change in GlobalVar.cache_state_changes:
        if change['level'] == 'L2':  # Filter by level
            print(f"L2 Change: {change}")

In [30]:
example_usage()


=== CACHE STATE CHANGES ===

Cycle 15: Core 0 L1 READ @ 2
  Set 0, Line 0:
    Valid: False → True
    Dirty: False → True
    Tag: None → 0
    Changes: valid: INVALID→VALID, dirty: CLEAN→DIRTY, tag: None→0

Cycle 31: Core 0 L1 READ @ 1
  Set 0, Line 0:
    Valid: True → True
    Dirty: True → False
    Tag: 0 → 0
    Changes: dirty: DIRTY→CLEAN

Cycle 31: Core 0 L1 READ @ 1
  Set 0, Line 1:
    Valid: False → True
    Dirty: False → True
    Tag: None → 1
    Changes: valid: INVALID→VALID, dirty: CLEAN→DIRTY, tag: None→1

Cycle 31: Core 0 L1 READ @ 1
  Set 3, Line 0:
    Valid: False → True
    Dirty: False → True
    Tag: None → 0
    Changes: valid: INVALID→VALID, dirty: CLEAN→DIRTY, tag: None→0

Cycle 39: Core 0 L1 READ @ 6
  Set 0, Line 0:
    Valid: True → True
    Dirty: False → True
    Tag: 0 → 0
    Changes: dirty: CLEAN→DIRTY

Cycle 39: Core anycore L2 READ @ 6
  Set 0, Line 0:
    Valid: False → True
    Dirty: False → True
    Tag: None → 0
    Changes: valid: INVALID→VA

In [54]:
exp.ratios_tab

array([[ 1., -0.,  1., -0.],
       [-0.,  1., -0.,  1.]])

In [12]:
len(exp.ddr_stats['row'])

4

In [37]:
exp.miss_ratio_global

np.float64(1.0)

In [13]:
import numpy as np

In [14]:
hits = np.zeros(exp.num_banks)
miss = np.zeros(exp.num_banks)
hits_tab = np.zeros((exp.num_rows,exp.num_banks))
miss_tab = np.zeros((exp.num_rows,exp.num_banks))

In [15]:
exp.ddr_stats

{'row': [0, 0, 1, 1],
 'bank': [0, 2, 3, 1],
 'status': ['ROW MISS', 'ROW MISS', 'ROW MISS', 'ROW MISS'],
 'core': [1, 1, 1, 1]}

In [50]:
for j in range(len(exp.ddr_stats['row'])):
    if exp.ddr_stats['status'][j]=='ROW MISS':
        miss[exp.ddr_stats['bank'][j]] +=1
        miss_tab[exp.ddr_stats['row'][j],exp.ddr_stats['bank'][j]] +=1
    else:
        hits[exp.ddr_stats['bank']] +=1
        hits_tab[exp.ddr_stats['row'][j],exp.ddr_stats['bank'][j]] +=1

In [55]:
exp.time_values

{'core0': [0, 10, 60], 'core1': [3, 50, 54, 15, 45, 95, 112]}

In [44]:
l1_conf = {'size': 32, 'line_size': 4, 'assoc': 2}
l2_conf = {'size': 1024, 'line_size': 4, 'assoc': 16}
# Create Core-specific Multi-Level Caches, connected to the shared L2
mem_core0 = MultiLevelCache(0, l1_conf, exp.mem_core0.l1.lower)

In [45]:
print(mem_core0.l1._index(0))
print(mem_core0.l1._index(17))

0
0


In [46]:
print(mem_core0.l1._tag(0))
print(mem_core0.l1._tag(17))

0
1


In [28]:
def print_cache_states_history(line_idx:int, set_idx:int):
    """Print the cache states history in a readable format"""
    print("\n=== CACHE STATES HISTORY ===")
    for state in GlobalVar.cache_states_history:
        
        print(f"\nCycle {state['cycle']}: Core {state['core_id']} {state['level']} {state['operation'].upper()} @ {state['address']}")
        
        for set_state in state['sets']:
            if set_state['set_index']!=set_idx:
                #print(set_state)
                continue
            set_index = set_state['set_index']
            print(f"  Set {set_index}:")
            
            for line_state in set_state['lines']:
                if line_state['line_index']!=line_idx:
                    continue
                status = "INVALID"
                if line_state['valid']:
                    status = "CLEAN" if not line_state['dirty'] else "DIRTY"
                
                tag_info = f"Tag: {line_state['tag']}" if line_state['tag'] is not None else "Tag: None"
                print(f"    Line {line_state['line_index']}: {status} | {tag_info}")

In [29]:
print_cache_states_history(0,0)


=== CACHE STATES HISTORY ===

Cycle 26: Core 0 L1 READ @ 16
  Set 0:
    Line 0: INVALID | Tag: None

Cycle 26: Core anycore L2 READ @ 16
  Set 0:
    Line 0: INVALID | Tag: None

Cycle 54: Core 0 L1 READ @ 8
  Set 0:
    Line 0: INVALID | Tag: None

Cycle 54: Core anycore L2 READ @ 8
  Set 0:
    Line 0: INVALID | Tag: None

Cycle 18: Core 1 L1 READ @ 9
  Set 0:
    Line 0: DIRTY | Tag: 1

Cycle 18: Core anycore L2 READ @ 9
  Set 0:
    Line 0: INVALID | Tag: None

Cycle 39: Core 1 L1 READ @ 11
  Set 0:
    Line 0: DIRTY | Tag: 1

Cycle 39: Core anycore L2 READ @ 11
  Set 0:
    Line 0: INVALID | Tag: None

Cycle 48: Core 1 L1 READ @ 2
  Set 0:
    Line 0: DIRTY | Tag: 1

Cycle 48: Core anycore L2 READ @ 2
  Set 0:
    Line 0: INVALID | Tag: None

Cycle 18: Core 1 L1 READ @ 9
  Set 0:
    Line 0: DIRTY | Tag: 1

Cycle 18: Core anycore L2 READ @ 9
  Set 0:
    Line 0: INVALID | Tag: None

Cycle 26: Core 0 L1 READ @ 16
  Set 0:
    Line 0: INVALID | Tag: None

Cycle 26: Core anycore L2